In [360]:
import numpy as np
from pc_kriging import PC_Kriging
from datetime import datetime
# import matplotlib.pyplot as plt
from scipy.stats import norm
from doepy import build
from scipy import optimize
import pandas as pd
from numpy import genfromtxt
import pickle

class Active_Training():

    def __init__(self, model=None, settings=None):
        if model is None:
            model = {"metamodel" : 'PCK'}
        assert "metamodel" in model and \
            "Missing model type"
        
        self.date_record = datetime.now().strftime("%Y_%m_%d_%H%M%S")

        #----------------------------------------------------------------------
        self.doe_limits = {}              #dict with marginals limits (for initial sampling)
        self.config = {}
        poly_type = []

        self.n_o = settings["passive_sampling"][0]        #DoE size
        dim = len(settings["marginals_R"][0])      #Input dimensions
        self.n_act = settings["active_sampling"][0]     #Number of active samples
        self.passive_strat = settings["passive_sampling"][1]

        self.xn_o = np.zeros((self.n_o, dim))      #normalized samples
        self.xr_o = np.zeros((self.n_o, dim))      #scaled samples
        self.yn_o = np.zeros((self.n_o))           #observations

        self.MaternCoef = settings["model"][0]
        self.p_max = settings["model"][1]
        #----------------------------------------------------------------------
        for margin in range (0, dim):
            doe_limits['x' + str (margin+1)] = [-1, 1]
            poly_type.append(settings["marginals_R"][0]['x' + str (margin + 1)][2])

        self.config["pol_type"] = poly_type

        #----------------------------------------------------------------------
        if model["metamodel"] == 'PCK':
            self.surrogate = PC_Kriging(self.config)

        # if settings["passive_sampling"][1] == 'LHS':   #Latin hypercube sampling
        #     Xdoe_N = build.space_filling_lhs( doe_limits , num_samples = self.n_o )
        Xdoe_N = self.passive_sampling(self.n_o)
        # # else WE SHOULD ADD 'RND' in case of random sampling  

        #----------------------------------------------------------------------
        for margin in range (0, dim):
            self.xn_o[:, margin] = Xdoe_N['x' + str (margin+1)]
            
            if self.config["pol_type"][margin] == 'hermite':
                mean_ = settings["marginals_R"][0]['x' + str (margin + 1)][0]
                var_ = settings["marginals_R"][0]['x' + str (margin + 1)][1]
                self.xr_o[:, margin] = self.surrogate.scalehermite( self.xn_o[:, margin], mean_, var_ )
            else:
                min_ = settings["marginals_R"][0]['x' + str (margin + 1)][0]
                max_ = settings["marginals_R"][0]['x' + str (margin + 1)][1]
                self.xr_o[:, margin] = self.surrogate.scalelegendre( self.xn_o[:, margin], min_, max_ )
        #----------------------------------------------------------------------
        self.yn_o = settings["active_sampling"][1](self.xr_o)

    def passive_sampling (self, n):
        if  self.passive_strat == 'LHS':   #Latin hypercube sampling

            return  build.space_filling_lhs( self.doe_limits , num_samples = n )
        
            # else WE SHOULD ADD 'RND' in case of random sampling  

    def active_sampling(self, number_experiments):
        if number_experiments is None:
            number_experiments = 1

        for experiments in range(number_experiments):     

            experiment_results = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
            
            for points in range(self.n_act):

                mse_results = np.zeros(self.p_max - 1)
                opt_length_it = np.zeros(self.p_max - 1)
                eloo_results = np.zeros(self.p_max - 1)
                
                mean_loo = np.zeros(len(xn))
                var_loo = np.zeros(len(xn))


    def EFF(self, u, v, z):
        zl=-2*v
        zh=2*v
        return ((u-z)*( 2*norm.cdf((z-u)/v) - norm.cdf((zl-u)/v) - norm.cdf((zh-u)/v)) 
                -(v)*( 2*norm.pdf((z-u)/v) - norm.pdf((zl-u)/v) - norm.pdf((zh-u)/v))  
                +(2*v)*(norm.cdf((zh-u)/v) - norm.cdf((zl-u)/v)))
    
    def U_function(self, u, v):
        return np.abs(u)/v
    
    def VoronoiCell(self, x,xn):   #given x [single value] return the index of the closest xn [array]
        dist = self.surrogate.distance(x.reshape(1,-1),xn)
        return np.argmin(dist)
    
    
    

In [359]:
training_settings["model"][1]

5

In [356]:
def LS1(x):                   #Definition of target limit state
    return x*np.sin(x)

model = {"metamodel" : 'PCK'}
training_settings = {"passive_sampling" : [20,'LHS'] , # initial samplings [n, dim, method]
                     "marginals_R" : [{'x1':[0, 15, 'legendre']}],   # {'x1':[Min, Max], 'x2':[mean, var]}  ( if Legendre , if Hermite)
                     "active_sampling" : [2, LS1, 'U'], #target settings [n_act, targetF, learningF]
                     "model": [5/2, 5]}                  #modelparameters, matern coeff, max polynomial degree

PCK1 = Active_Training(model, training_settings)

In [351]:
n_act = 5
experiments = 1

def launch_training(number_experiments):
    if number_experiments is None:
        number_experiments = 1

    for experiments in range(number_experiments):     

        experiment_results = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

        for points in range(n_act):

            mse_results = np.zeros(p_max-1)
            opt_length_it = np.zeros(p_max-1)
            eloo_results = np.zeros(p_max-1)
        
        


In [283]:
training_settings["marginals_R"][0]['x' + str (2)][2]

'hermite'

In [199]:
training_settings["marginals_R"][0]

{'x1': [0, 15, 'legendre']}

In [306]:
#active training

def launch_training(settings):

    n = settings["passive_sampling"][0]        #DoE size
    dim = len(settings["marginals_R"][0])      #Input dimensions
    n_act = settings["active_sampling"][0]     #Number of active samples

    xn = np.zeros((n, dim))      #normalized samples
    xr = np.zeros((n, dim))      #scaled samples
    yn = np.zeros((n))           #observations
    #----------------------------------------------------------------------
    doe_limits = {}              #dict with marginals limits (for initial sampling)
    config = {}
    poly_type = []
    # marginals_N = {}             #dict for marginals normaliz parameters following 'config'
    #----------------------------------------------------------------------
    for margin in range (0, dim):
        doe_limits['x' + str (margin+1)] = [-1, 1]
        poly_type.append(training_settings["marginals_R"][0]['x' + str (margin + 1)][2])

    config["pol_type"] = poly_type

    if settings["passive_sampling"][1] == 'LHS':
        Xdoe_N = build.space_filling_lhs( doe_limits , num_samples = n )
    #----------------------------------------------------------------------
    for margin in range (0, dim):
        xn[:, margin] = Xdoe_N['x' + str (margin+1)]
        
        if config["pol_type"][margin] == 'hermite':
            mean_ = settings["marginals_R"][0]['x' + str (margin + 1)][0]
            var_ = settings["marginals_R"][0]['x' + str (margin + 1)][1]
            xr[:, margin] = PCK1.surrogate.scalehermite( xn[:, margin], mean_, var_ )
        else:
            min_ = settings["marginals_R"][0]['x' + str (margin + 1)][0]
            max_ = settings["marginals_R"][0]['x' + str (margin + 1)][1]
            xr[:, margin] = PCK1.surrogate.scalelegendre( xn[:, margin], min_, max_ )
    #----------------------------------------------------------------------
    yn = settings["active_sampling"][1](xr)

    # for points in range(n_act):


In [290]:
config1 = {}

In [299]:
poly_type = []

In [302]:
poly_type.append('legendre')

In [304]:
poly_type.append('hermite')

In [305]:
poly_type

['legendre', 'hermite']

{'pol_type': ['legendre']}

In [254]:
training_settings["active_sampling"][1]

<function __main__.LS1(x)>

In [307]:
launch_training(training_settings)

{'pol_type': ['legendre', 'hermite']}

In [212]:
config

{'pol_type': ['legendre']}

In [192]:
yn

NameError: name 'yn' is not defined

In [142]:
doe_limits = {}
marginals_N = {}

for margin in range (0, len(config["pol_type"])):
    doe_limits['x' + str (margin+1)] = [-1, 1]

    if config["pol_type"][margin] == 'hermite':
        marginals_N['x' + str (margin+1)] = [0 , 1]
    else:
        marginals_N['x' + str (margin+1)] = [-1 , 1]

In [19]:
PCK1.surrogate.polynomials[0]

<bound method PC_Kriging.hermite of <pc_kriging.PC_Kriging object at 0x0000025AFE471688>>

In [6]:
xr1 = np.array( [9.60173493460726,
14.0942931914357,
13.3422784718819,
6.92708483258534,
4.44528591511115,
7.66438265483689,
0.0286187144815328,
5.91008701677294,
10.6641810829072,
2.13464775089995]).reshape(-1,1)

xn1 = PCK1.surrogate.LinearNorm(xr1, 0.0, 15.0, -1.0, 1.0)

y1 = LS1(xr1) 